# Dependencies

In [10]:
import pandas as pd
import numpy as np
import time
import threading

# Generating Graph

In [11]:
import random

def generate_graph_entries(num_vertices):
    vertices = [chr(ord('A') + i) for i in range(num_vertices)]
    edges_data = {'source': [], 'destination': [], 'weight': []}

    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            edges_data['source'].append(vertices[i])
            edges_data['destination'].append(vertices[j])
            edges_data['weight'].append(random.randint(1, 10))

    edges_df = pd.DataFrame(edges_data)

    return vertices, edges_df

# Prims Algorithm

1. Initialize an empty set to store selected vertices (MSTSet) and an empty list to store minimum spanning tree edges (mstEdges).
2. Randomly choose a starting vertex to begin the algorithm.
3. Add the starting vertex to the MSTSet.
4. While MSTSet does not contain all vertices:
      a. Find the minimum-weight edge that connects a vertex in MSTSet to a vertex not in MSTSet.
      b. Add the destination vertex of the minimum-weight edge to MSTSet.
      c. Add the minimum-weight edge to mstEdges.
5. Return mstEdges, which represents the minimum spanning tree.

In [12]:
class PrimMST:
    def __init__(self, vertices, edges):
        self.vertices = vertices
        self.edges = edges
        self.mst = []

    def prim(self):
        selected_vertices = set()
        start_vertex = self.vertices[0]
        selected_vertices.add(start_vertex)

        while len(selected_vertices) < len(self.vertices):
            min_weight = float('inf')
            min_edge = None
            for vertex in selected_vertices:
                edges_from_vertex = self.edges[(self.edges['source'] == vertex) | (self.edges['destination'] == vertex)]
                for _, edge in edges_from_vertex.iterrows():
                    if (edge['source'] in selected_vertices and edge['destination'] not in selected_vertices
                        and edge['weight'] < min_weight):
                        min_weight = edge['weight']
                        min_edge = (edge['source'], edge['destination'], min_weight)
                    elif (edge['destination'] in selected_vertices and edge['source'] not in selected_vertices
                          and edge['weight'] < min_weight):
                        min_weight = edge['weight']
                        min_edge = (edge['source'], edge['destination'], min_weight)

            self.mst.append(min_edge)
            selected_vertices.add(min_edge[0])
            selected_vertices.add(min_edge[1])

        return self.mst

# Example usage

num_vertices = 7
vertices, edges_df = generate_graph_entries(num_vertices)
prim = PrimMST(vertices, edges_df)
minimum_spanning_tree = prim.prim()
print("Minimum Spanning Tree:")
for edge in minimum_spanning_tree:
    print(edge)

Minimum Spanning Tree:
('A', 'D', 4)
('B', 'D', 4)
('D', 'F', 4)
('F', 'G', 1)
('E', 'G', 2)
('C', 'G', 4)


# Kruskal Algorithm

1. Initialize an empty list to store the minimum spanning tree edges (mstEdges).
2. Sort all the edges of the graph in non-decreasing order of their weight.
3. Initialize a disjoint set data structure to keep track of connected components.
4. For each vertex in the graph, create a set containing only that vertex.
5. For each edge (u, v) in the sorted edges:
      a. If the sets containing u and v are not the same (i.e., adding the edge does not create a cycle):
           i. Add the edge (u, v) to mstEdges.
           ii. Merge the sets containing u and v.
6. Return mstEdges, which represents the minimum spanning tree.

In [13]:
class KruskalMST:
    def __init__(self, vertices, edges):
        self.vertices = vertices
        self.edges = edges
        self.parent = {}
        self.mst = []

    def make_set(self, vertex):
        self.parent[vertex] = vertex

    def find(self, vertex):
        if self.parent[vertex] != vertex:
            self.parent[vertex] = self.find(self.parent[vertex])
        return self.parent[vertex]

    def union(self, vertex1, vertex2):
        root1 = self.find(vertex1)
        root2 = self.find(vertex2)
        self.parent[root1] = root2

    def kruskal(self):
        for vertex in self.vertices:
            self.make_set(vertex)

        sorted_edges = self.edges.sort_values(by='weight')

        for _, edge in sorted_edges.iterrows():
            source, destination, weight = edge
            if self.find(source) != self.find(destination):
                self.mst.append((source, destination, weight))
                self.union(source, destination)

        return self.mst

# Example usage
num_vertices = 7
vertices, edges_df = generate_graph_entries(num_vertices)
kruskal = KruskalMST(vertices, edges_df)
minimum_spanning_tree = kruskal.kruskal()
print("Minimum Spanning Tree:")
for edge in minimum_spanning_tree:
    print(edge)

Minimum Spanning Tree:
('B', 'G', 1)
('D', 'E', 2)
('A', 'F', 3)
('C', 'D', 4)
('F', 'G', 4)
('E', 'G', 5)


# Running algorithms in parallel

In [9]:

def run_kruskal(vertices, edges_df, result_list, time_list):
    start_time = time.time()
    kruskal = KruskalMST(vertices, edges_df)
    minimum_spanning_tree = kruskal.kruskal()
    end_time = time.time()
    result_list.append(minimum_spanning_tree)
    time_list.append(end_time - start_time)

def run_prim(vertices, edges_df, result_list, time_list):
    start_time = time.time()
    prim = PrimMST(vertices, edges_df)
    minimum_spanning_tree = prim.prim()
    end_time = time.time()
    result_list.append(minimum_spanning_tree)
    time_list.append(end_time - start_time)

num_vertices = 5
vertices, edges_df = generate_graph_entries(num_vertices)

kruskal_result = []
prim_result = []
kruskal_time = []
prim_time = []

# Create threads for Kruskal's and Prim's algorithms
thread_kruskal = threading.Thread(target=run_kruskal, args=(vertices, edges_df, kruskal_result, kruskal_time))
thread_prim = threading.Thread(target=run_prim, args=(vertices, edges_df, prim_result, prim_time))

# Start the threads
thread_kruskal.start()
thread_prim.start()

# Wait for the threads to finish
thread_kruskal.join()
thread_prim.join()

print("Kruskal's Minimum Spanning Tree:")
for edge in kruskal_result[0]:
    print(edge)

print("\nPrim's Minimum Spanning Tree:")
for edge in prim_result[0]:
    print(edge)

print("\nTime taken by Kruskal's algorithm:", kruskal_time[0], "seconds")
print("Time taken by Prim's algorithm:", prim_time[0], "seconds")

Kruskal's Minimum Spanning Tree:
('C', 'D', 1)
('D', 'E', 1)
('A', 'E', 5)
('A', 'B', 7)

Prim's Minimum Spanning Tree:
('A', 'E', 5)
('D', 'E', 1)
('C', 'D', 1)
('A', 'B', 7)

Time taken by Kruskal's algorithm: 0.0019989013671875 seconds
Time taken by Prim's algorithm: 0.013001680374145508 seconds
